In [1]:
include("../../../julia/FNC_init.jl")

  Activating 

project at `~/Documents/GitHub/fnc`


[**Demo %s**](#demo-structure-linalg)

The following generates a random sparse matrix with prescribed eigenvalues.

In [2]:
using SparseArrays
n = 4000
density = 4e-4
λ = @. 1 + 1 / (1:n)   # exact eigenvalues
A = FNC.sprandsym(n, density, λ);

```{index} ! Julia; eigs
```

The `eigs` function from `Arpack` finds a small number eigenvalues meeting some criterion. First, we ask for the 5 of largest (complex) magnitude using `which=:LM`.

In [3]:
using Arpack
λmax, V = eigs(A, nev=5, which=:LM)    # Largest Magnitude
fmt = ft_printf("%20.15f")
pretty_table([λmax λ[1:5]], header=["found", "exact"], formatters=fmt)

┌──────────────────────┬──────────────────────┐
│                found │                exact │
├──────────────────────┼──────────────────────┤
│    2.000000000000000 │    2.000000000000000 │
│    1.500000000000002 │    1.500000000000000 │
│    1.333333333333335 │    1.333333333333333 │
│    1.250000000000002 │    1.250000000000000 │
│    1.200000000000002 │    1.200000000000000 │
└──────────────────────┴──────────────────────┘


Now we find the 5 closest to the value 1 in the complex plane, via `sigma=1`.

In [4]:
λ1, V = eigs(A, nev=5, sigma=1)    # closest to sigma
data = [λ1 λ[end:-1:end-4]]
pretty_table(data, header=["found", "exact"], formatters=fmt)

┌──────────────────────┬──────────────────────┐
│                found │                exact │
├──────────────────────┼──────────────────────┤
│    1.000250000000000 │    1.000250000000000 │
│    1.000250062515629 │    1.000250062515629 │
│    1.000250125062531 │    1.000250125062531 │
│    1.000250187640731 │    1.000250187640731 │
│    1.000250250250250 │    1.000250250250250 │
└──────────────────────┴──────────────────────┘


```{index} Julia; \\
```

The time needed to solve a sparse linear system is not easy to predict unless you have some more information about the matrix. But it will typically be orders of magnitude faster than the dense version of the same problem.

In [5]:
x = @. 1 / (1:n);
b = A * x;

In [6]:
norm(x - A \ b);  # force compilation
t = @elapsed sparse_err = norm(x - A \ b)
println("Time for sparse solve: $t")

Time for sparse solve: 0.000668625


In [7]:
D = Matrix(A)  # convert to regular matrix
norm(x - D \ b);
t = @elapsed dense_err = norm(x - D \ b)
println("Time for dense solve: $t")

Time for dense solve: 0.328220208


In [8]:
@show sparse_err;
@show dense_err;

sparse_err = 8.437600965970206e-17
dense_err = 4.979663977679232e-18
